In [1]:
!yolo checks

Ultralytics YOLOv8.0.230 🚀 Python-3.11.5 torch-2.2.0.dev20230911 CPU (Apple M2)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 251.6/460.4 GB disk)

OS                  macOS-14.2.1-arm64-arm-64bit
Environment         Darwin
Python              3.11.5
Install             git
RAM                 16.00 GB
CPU                 Apple M2
CUDA                None

matplotlib          ✅ 3.7.2>=3.3.0
numpy               ✅ 1.24.4>=1.22.2
opencv-python       ✅ 4.8.1.78>=4.6.0
pillow              ✅ 10.0.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.2>=1.4.1
torch               ✅ 2.2.0.dev20230911>=1.8.0
torchvision         ✅ 0.17.0.dev20230911>=0.9.0
tqdm                ✅ 4.66.1>=4.64.0
pandas              ✅ 2.1.0>=1.1.4
seaborn             ✅ 0.12.2>=0.11.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1


In [107]:
import ast
import numpy as np
import os
import pandas as pd
import pybboxes as pbx
import shutil
from sklearn.model_selection import train_test_split

In [109]:
DIR = "img1"

LABELS_CSV = f"{DIR}/annotations.csv"

LABELS_DF = pd.read_csv(LABELS_CSV)

OUTPUT_DIR = f"{DIR}/output"
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [110]:
LABELS_DF

,id,image_id,geometry,class
0,1,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(135, 522), (245, 522), (245, 600), (135, 600...",Airplane
1,2,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1025, 284), (1125, 284), (1125, 384), (1025,...",Airplane
2,3,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1058, 1503), (1130, 1503), (1130, 1568), (10...",Airplane
3,4,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(813, 1518), (885, 1518), (885, 1604), (813, ...",Airplane
4,5,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(594, 938), (657, 938), (657, 1012), (594, 10...",Airplane
...,...,...,...,...
3420,3421,48ef8e15-a43c-406b-9d3c-e815164b96d1.jpg,"[(1059, 393), (1187, 393), (1187, 522), (1059,...",Airplane
3421,3422,48ef8e15-a43c-406b-9d3c-e815164b96d1.jpg,"[(932, 355), (1028, 355), (1028, 461), (932, 4...",Airplane
3422,3423,48ef8e15-a43c-406b-9d3c-e815164b96d1.jpg,"[(2461, 244), (2560, 244), (2560, 345), (2461,...",Truncated_airplane
3423,3424,48ef8e15-a43c-406b-9d3c-e815164b96d1.jpg,"[(1820, 344), (1937, 344), (1937, 443), (1820,...",Airplane


In [111]:
def get_bbox(row):
    arr = ast.literal_eval(row["geometry"])
    x, y = zip(*arr) 
    xmin = np.min(x)
    ymin = np.min(y)
    xmax = np.max(x)
    ymax = np.max(y)
    return [xmin, ymin, xmax, ymax]

def voc_to_yolo(row):
    return pbx.convert_bbox(row["vocbbox"],
                            from_type="voc",
                            to_type="yolo",
                            image_size=(2560,2560))

LABELS_DF["vocbbox"] = LABELS_DF.apply(get_bbox, axis=1)
LABELS_DF["yolobbox"] = LABELS_DF.apply(voc_to_yolo, axis=1)
LABELS_DF["image_id"] = LABELS_DF["image_id"].str.replace(r"\.jpg$", "", regex=True)

In [112]:
LABELS_DF

,id,image_id,geometry,class,vocbbox,yolobbox
0,1,4f833867-273e-4d73-8bc3-cb2d9ceb54ef,"[(135, 522), (245, 522), (245, 600), (135, 600...",Airplane,"[135, 522, 245, 600]","(0.07421875, 0.219140625, 0.04296875, 0.03046875)"
1,2,4f833867-273e-4d73-8bc3-cb2d9ceb54ef,"[(1025, 284), (1125, 284), (1125, 384), (1025,...",Airplane,"[1025, 284, 1125, 384]","(0.419921875, 0.13046875, 0.0390625, 0.0390625)"
2,3,4f833867-273e-4d73-8bc3-cb2d9ceb54ef,"[(1058, 1503), (1130, 1503), (1130, 1568), (10...",Airplane,"[1058, 1503, 1130, 1568]","(0.42734375, 0.5998046875, 0.028125, 0.025390625)"
3,4,4f833867-273e-4d73-8bc3-cb2d9ceb54ef,"[(813, 1518), (885, 1518), (885, 1604), (813, ...",Airplane,"[813, 1518, 885, 1604]","(0.331640625, 0.609765625, 0.028125, 0.03359375)"
4,5,4f833867-273e-4d73-8bc3-cb2d9ceb54ef,"[(594, 938), (657, 938), (657, 1012), (594, 10...",Airplane,"[594, 938, 657, 1012]","(0.2443359375, 0.380859375, 0.024609375, 0.028..."
...,...,...,...,...,...,...
3420,3421,48ef8e15-a43c-406b-9d3c-e815164b96d1,"[(1059, 393), (1187, 393), (1187, 522), (1059,...",Airplane,"[1059, 393, 1187, 522]","(0.438671875, 0.1787109375, 0.05, 0.050390625)"
3421,3422,48ef8e15-a43c-406b-9d3c-e815164b96d1,"[(932, 355), (1028, 355), (1028, 461), (932, 4...",Airplane,"[932, 355, 1028, 461]","(0.3828125, 0.159375, 0.0375, 0.04140625)"
3422,3423,48ef8e15-a43c-406b-9d3c-e815164b96d1,"[(2461, 244), (2560, 244), (2560, 345), (2461,...",Truncated_airplane,"[2461, 244, 2560, 345]","(0.9806640625, 0.1150390625, 0.038671875, 0.03..."
3423,3424,48ef8e15-a43c-406b-9d3c-e815164b96d1,"[(1820, 344), (1937, 344), (1937, 443), (1820,...",Airplane,"[1820, 344, 1937, 443]","(0.7337890625, 0.1537109375, 0.045703125, 0.03..."


In [113]:
OUTPUT_DIR = f"{DIR}/images"

LABELS_DF_GROUPED = LABELS_DF.groupby("image_id")

for filename, group in LABELS_DF_GROUPED:
    OUTPUT_FILE_PATH = os.path.join(OUTPUT_DIR, f"{filename}.txt")
    with open(OUTPUT_FILE_PATH, "w") as file:
        for index, row in group.iterrows():
            yolobox_vals = " ".join(map(str, row.iloc[-1]))
            file.write(f"0 {yolobox_vals}\n")

In [115]:
def split_images_and_text(source_folder, train_folder, valid_folder, split_ratio=0.8, random_seed=42):
    # Ensure the source folder exists
    if not os.path.exists(source_folder):
        print(f"Error: Source folder '{source_folder}' not found.")
        return

    # Create train and validation folders if they don't exist
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(valid_folder, exist_ok=True)

    # Get a list of all image files in the source folder
    image_files = [f for f in os.listdir(source_folder) if f.lower().endswith('.jpg')]

    # Split the images into train and validation sets
    train_images, valid_images = train_test_split(image_files, test_size=(1 - split_ratio), random_state=random_seed)

    # Copy images and corresponding text files to the train folder
    for image in train_images:
        # Copy image file
        src_image_path = os.path.join(source_folder, image)
        dest_image_path = os.path.join(train_folder, image)
        shutil.copy(src_image_path, dest_image_path)

        # Copy corresponding text file
        text_file = image.replace('.jpg', '.txt')
        src_text_path = os.path.join(source_folder, text_file)
        dest_text_path = os.path.join(train_folder, text_file)
        shutil.copy(src_text_path, dest_text_path)

    # Copy images and corresponding text files to the validation folder
    for image in valid_images:
        # Copy image file
        src_image_path = os.path.join(source_folder, image)
        dest_image_path = os.path.join(valid_folder, image)
        shutil.copy(src_image_path, dest_image_path)

        # Copy corresponding text file
        text_file = image.replace('.jpg', '.txt')
        src_text_path = os.path.join(source_folder, text_file)
        dest_text_path = os.path.join(valid_folder, text_file)
        shutil.copy(src_text_path, dest_text_path)

    print(f"Split {len(image_files)} pairs of images and text files into {len(train_images)} for training and {len(valid_images)} for validation.")

# Example usage
source_folder = f'{DIR}/images'
train_folder = f'{DIR}/train'
valid_folder = f'{DIR}/valid'

split_images_and_text(source_folder, train_folder, valid_folder, split_ratio=0.8, random_seed=42)


Split 103 pairs of images and text files into 82 for training and 21 for validation.


In [116]:
CONFIG = """
train: /Users/user/github/aircraft/img1/train
val: /Users/user/github/aircraft/img1/valid
names:
  0: plane
"""

with open(os.path.join(OUTPUT_DIR, "data.yaml"), "w") as f:
    f.write(CONFIG)

In [118]:
HOME = OUTPUT_DIR
!yolo task=detect mode=train model=yolov8n.pt data={HOME}/data.yaml epochs=10 batch=16 imgsz=1024 device=cpu

New https://pypi.org/project/ultralytics/8.0.231 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.230 🚀 Python-3.11.5 torch-2.2.0.dev20230911 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=img1/images/data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sa

In [ ]:
!yolo task=detect mode=train resume model=/opt/homebrew/runs/detect/train4/weights/last.pt data={HOME}/data.yaml epochs=10 batch=16 imgsz=1024 device=cpu